In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

from sklearn.metrics import make_scorer, f1_score

np.random.seed(1234)

num_figures = 10000

In [2]:
%%time
features = []

target_nx = 200
target_ny = 300

extract_ratio = 1e-2

random_part = np.random.rand(target_nx*target_ny) < extract_ratio
random_part = np.array([random_part]*3).reshape(target_nx, target_ny, 3)
#random_part = random_part.reshape(target_nx, target_ny, 3)

#im_sum = np.zeros([target_nx, target_ny, 3])

for i in range(num_figures):
    im = plt.imread('task4_be9ai3nsdj/food/' + f'{i:05d}' +'.jpg')
    
    nx, ny, nc = np.shape(im)
    
    start_idx = int((nx - target_nx)/2)
    end_idx = target_nx + start_idx
    
    start_idy = int((ny - target_ny)/2)
    end_idy = target_ny + start_idy
        
    im_cropped = im[start_idx:end_idx, start_idy:end_idy, :]
    
    features.append(im_cropped[random_part])

CPU times: user 25.9 s, sys: 485 ms, total: 26.3 s
Wall time: 30.6 s


In [3]:
triplets = pd.read_csv('task4_be9ai3nsdj/train_triplets.txt', header=None, delimiter=' ', dtype=str).to_numpy()
triplets_id = pd.read_csv('task4_be9ai3nsdj/train_triplets.txt', header=None, delimiter=' ', dtype=int).to_numpy()

In [4]:
num_feature = len(features[0])
num_obsv = len(triplets_id)

triplets_id_appended = np.append(triplets_id, np.array([triplets_id[:,0], triplets_id[:,2], triplets_id[:,1]]).T, axis=0)

train_observation = np.zeros(len(triplets_id_appended))
train_observation[:num_obsv] = 1

In [5]:
print(num_feature)

1830


In [6]:
feature_transformed = []

for id_list in triplets_id_appended:
    feature_transformed.append([features[idx] for idx in id_list][0])
    
feature_transformed = np.array(feature_transformed)

In [7]:
score_func = make_scorer(f1_score)

In [ ]:
%%time
model = MLPClassifier(hidden_layer_sizes=100, activation='identity'
                      , solver='adam', max_iter=5000, tol=1e-5, alpha=1e-7)

scores = cross_val_score(model, feature_transformed, train_observation, cv=5,
                         scoring=score_func, n_jobs=7)

print(scores)